In [1]:
import rootutils
rootutils.setup_root(".", indicator=".project-root", pythonpath=True, cwd=True)

PosixPath('/home/dgcnz/development/thesis/PART')

In [2]:
import pytest
from PIL import Image
import torchvision.transforms.v2.functional as TTFv2
import hydra
from omegaconf import OmegaConf
import torch
from src.models.part_vit_module import PARTViTModule
import lightning as L
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from copy import deepcopy
# import plotly dash
from torch.profiler import ProfilerActivity
from torch.profiler import record_function




In [3]:
def get_cfg():
    with hydra.initialize(version_base=None, config_path="../../configs"):
        cfg = hydra.compose(
            config_name="train.yaml",
            overrides=[
                "experiment=hs_part_im1k_pairdiff_mlp",
                "model.sample_mode=ongrid",
                "model.compile=False",
                "model/criterion_fn=l1",
                "model.net.backbone.patch_size=16",
                # "model/pair_sampler=random",
                # "model.pair_sampler.n_pairs=4000",
            ],
        )
    try:
        OmegaConf.register_new_resolver("eval", eval)
    except:
        pass
    return cfg

cfg = get_cfg()   

In [4]:
module: PARTViTModule = hydra.utils.instantiate(cfg.model)

/home/dgcnz/development/thesis/PART/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.


In [5]:
batch_size = 16
x: torch.Tensor = torch.randn(batch_size, 3, 224, 224)

In [6]:
module=module.cuda()
x=x.cuda()

In [7]:
# track memory snapshot
# torch.cuda.memory._record_memory_history(max_entries=100000)
# """
#    for _ in range(5):
#        pred = model(inputs)
#        loss_fn(pred, labels).backward()
#        optimizer.step()
#        optimizer.zero_grad(set_to_none=True)
# """
# optimizer = torch.optim.AdamW(module.parameters(), lr=1e-3)
# for _ in range(5):
#     out = module.model_step({"image":x})
#     loss: torch.Tensor = out["loss"]
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad(set_to_none=True)
# 
# try:
#     torch.cuda.memory._dump_snapshot(f"mem.pickle")
# except Exception as e:
#     print(f"Failed to capture memory snapshot {e}")
# 
# # Stop recording memory snapshot history.
# torch.cuda.memory._record_memory_history(enabled=None)
# 

In [8]:
# (c) Meta Platforms, Inc. and affiliates. 
import logging
import socket
from datetime import datetime, timedelta

import torch

from torch.autograd.profiler import record_function
from torchvision import models

logging.basicConfig(
   format="%(levelname)s:%(asctime)s %(message)s",
   level=logging.INFO,
   datefmt="%Y-%m-%d %H:%M:%S",
)
logger: logging.Logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)

TIME_FORMAT_STR: str = "%b_%d_%H_%M_%S"

def trace_handler(prof: torch.profiler.profile):
   # Prefix for file names.
   host_name = socket.gethostname()
   timestamp = datetime.now().strftime(TIME_FORMAT_STR)
   file_prefix = f"{host_name}_{timestamp}"

   # Construct the trace file.
   prof.export_chrome_trace(f"{file_prefix}.json.gz")

   # Construct the memory timeline file.
   prof.export_memory_timeline(f"{file_prefix}.html", device="cuda:0")

with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    # schedule=torch.profiler.schedule(wait=0, warmup=0, active=6, repeat=1),
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
    with_modules=True,
    experimental_config=torch._C._profiler._ExperimentalConfig(verbose=True)
    # on_trace_ready=trace_handler,
) as prof:
    module.eval()
    with torch.no_grad():
        out = module.model_step({"image": x})  # Forward pass
    # with record_function("## forward ##"):
    #    out = module.model_step({"image": x})  # Forward pass
    #    loss = out["loss"]                    # Get loss from output dict
    # with record_function("## backward ##"):
    #     loss.backward()                       # Backward pass
    # with record_function("## optimizer ##"):
    #     optimizer.step()                      # Optimizer step
    #     optimizer.zero_grad(set_to_none=True) # Zero grad
# prof.export_memory_timeline(f"profile.html", device="cuda:0")


/home/dgcnz/development/thesis/PART/.venv/lib/python3.10/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3611.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
import time
start_time = time.time()

In [10]:
prof.events()[1000].__dict__

{'id': 575,
 'node_id': -1,
 'name': 'aten::linear',
 'trace_name': 'aten::linear',
 'time_range': <torch.autograd.profiler_util.Interval at 0x79f408075c90>,
 'thread': 1,
 'fwd_thread': 0,
 'kernels': [],
 'count': 1,
 'cpu_children': [<FunctionEvent id=576 name=aten::reshape device_type=DeviceType.CPU node_id=-1 cpu_time=0.892us start_us=123462.628 end_us=123463.52 cpu_children=[577] cuda_time=0.000us name=aten::reshape thread=1 input_shapes=[[16, 196, 3072], []] cpu_memory_usage=0 cuda_memory_usage=0 is_async=False is_remote=False seq_nr=-1 is_legacy=False>,
  <FunctionEvent id=578 name=aten::t device_type=DeviceType.CPU node_id=-1 cpu_time=1.764us start_us=123463.88 end_us=123465.644 cpu_children=[579] cuda_time=0.000us name=aten::t thread=1 input_shapes=[[768, 3072]] cpu_memory_usage=0 cuda_memory_usage=0 is_async=False is_remote=False seq_nr=-1 is_legacy=False>,
  <FunctionEvent id=581 name=aten::addmm device_type=DeviceType.CPU node_id=-1 cpu_time=16.001us start_us=123472.326 en

In [11]:
stack=['<built-in method unbind of Tensor object at 0x7e9491e75490>',
  'timm/models/vision_transformer.py(85): forward',
  'torch/nn/modules/module.py(1740): _call_impl',
  'nn.Module: Attention_6',
  'timm/models/vision_transformer.py(164): forward',
  'torch/nn/modules/module.py(1740): _call_impl',
  'nn.Module: Block_6',
  'torch/nn/modules/container.py(248): forward',
  'torch/nn/modules/module.py(1740): _call_impl',
  'nn.Module: Sequential_0',
  'timm/models/vision_transformer.py(802): forward_features',
  'timm/models/vision_transformer.py(828): forward',
  'torch/nn/modules/module.py(1740): _call_impl',
  'nn.Module: VisionTransformer_0',
  'src/models/components/part_vit.py(75): forward',
  'torch/nn/modules/module.py(1740): _call_impl',
  'nn.Module: PARTViT_0',
  'src/models/part_vit_module.py(147): forward',
  'src/models/part_vit_module.py(177): model_step',
  '/tmp/ipykernel_161062/3606682951.py(33): <module>',
  'IPython/core/interactiveshell.py(3577): run_code',
  'IPython/core/interactiveshell.py(3517): run_ast_nodes',
  'IPython/core/interactiveshell.py(3334): run_cell_async',
  'IPython/core/async_helpers.py(128): _pseudo_sync_runner',
  'IPython/core/interactiveshell.py(3130): _run_cell',
  'IPython/core/interactiveshell.py(3075): run_cell',
  'ipykernel/zmqshell.py(549): run_cell',
  'ipykernel/ipkernel.py(449): do_execute',
  'ipykernel/kernelbase.py(778): execute_request',
  'ipykernel/ipkernel.py(362): execute_request',
  'ipykernel/kernelbase.py(437): dispatch_shell',]

# parse all "nn.Module"
import re
def parse_stack(stack):
    pat = re.compile(r"nn.Module: ([\w\d_]+)")
    modules = [ m.group(1) for m in [pat.match(s) for s in stack] if m is not None]
    # now return module names with slashes, example: PARTViT_0/VisonTransformer_0/Sequential_0/Block_6/Attention_6
    return "/".join(modules[::-1])


In [12]:
evs[0].time_range.start 

NameError: name 'evs' is not defined

In [14]:
evs = [ev for ev in prof.events() if "part_vit.py" in "\n".join(ev.stack)]
# [ev.stack[:6] for ev in evs]
# plot the memory usa
import plotly.express as px
import pandas as pd

# plot timeline with color coding for memory usage
from datetime import datetime

df = pd.DataFrame(
    [
        {
            # "cpu_start": datetime.fromtimestamp(start_time + ev.time_range.start / 1e6),
            # "cpu_end": datetime.fromtimestamp(start_time + ev.time_range.end / 1e6),
            "cpu_start": ev.time_range.start / 1e3,
            "cpu_end": ev.time_range.end / 1e3,
            "name": ev.name,
            "id": ev.id,
            "memory": ev.device_memory_usage / 1024**3,
            "stack": "\n".join(ev.stack[:5]),
            "module": parse_stack(ev.stack),
        }
        for ev in evs
    ]
)
df["main_module"] = df["module"].apply(
    lambda x: x.split("/")[1] if len(x.split("/")) > 1 else x
)
# df['delta'] = df['Finish'] - df['Start']
df["delta"] = df["cpu_end"] - df["cpu_start"]
df = df.sort_values("cpu_start")


# Create cumulative memory usage
df["cumulative_memory"] = df["memory"].cumsum()


# fig = px.timeline(evs, x_start="cpu_start", x_end="cpu_end", y="name", color="device", title="Memory Usage Timeline")
# fig = px.timeline(
#     df,
#     x_start="cpu_start",
#     x_end="cpu_end",
#     y="main_module",
#     color="memory",
#     title="Memory Usage Timeline",
#     hover_data="module",
# )  # , color_continuous_scale=px.colors.sequential.Teal)
# fig.update_yaxes(visible=False)
#
# fig.update_yaxes(autorange="reversed")
#
# fig.layout.xaxis.type = 'linear'
# make gantt instead of timeline
fig_timeline = px.timeline(
    df,
    x_start="cpu_start",
    x_end="cpu_end",
    y="name",
    color="memory",
    title="Memory Usage Timeline",
    hover_data="module",
    color_continuous_scale=px.colors.sequential.BuGn,
)
fig_timeline.update_yaxes(visible=False)
fig_timeline.layout.xaxis.type = "linear"
fig_timeline.data[0].x = df.delta.tolist()

# # Create the plot
fig_area = px.area(
    df,
    x="cpu_start",
    y="cumulative_memory",
    title="GPU Memory Usage Over Time",
    labels={"timestamp": "Time", "cumulative_memory": "Memory Usage (GB)"},
)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# i want to overlay the timeline and the area plot that is, both in the same figure
# it needs to be a timeline

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
timeline_trace = fig.add_trace(fig_timeline.data[0], row=1, col=1)
area_trace = fig.add_trace(fig_area.data[0], row=2, col=1)

# colors = 
max_mem = df.memory.max()
# timeblock_color = [px.colors.sequential.RdBu_r[int(m)] for m in df.memory]
# timeblock_color = [px.colors.sequential.Plotly3[int(9*m/max_mem)] for m in df.memory]

# invisible y acis for the timeline
fig.update_yaxes(visible=False, row=1, col=1)
# update y axis to 0 to 10 for the area plot
fig.update_yaxes(range=[0, 10], row=2, col=1)
# update colorscheme of timeline to Teal
# timeline_trace
# fig.update_traces(marker=dict(color=df["memory"], colorscale=px.colors.sequential.BuGn), selector=dict(type="bar"))
# fig.update_traces(marker=dict(color=timeblock_color, ), selector=dict(type="bar"), overwrite=True)
# fig.update_layout(coloraxis_colorbar=dict(title="Memory (GB)"))
# add colorbar to legend
fig.update_layout(coloraxis_colorbar=dict(title="Memory (GB)"))

# add vertical lines for each main_module's start_time with its main_module as label
# main_df = df.groupby("main_module").first()
# for i, row in main_df.iterrows():
#     fig.add_vline(x=row.cpu_start, line_width=1, line_dash="dot", line_color="red", row=1, col=1)
#     fig.add_vline(x=row.cpu_start, line_width=1, line_dash="dot", line_color="red", row=2, col=1)

# main_df = df.groupby("main_module").first()
# for i, row in main_df.iterrows():
#     # set annotation text at 45 degree angle
#     # fig.add_vline(x=row.cpu_start, line_width=1, line_dash="dot", line_color="red", row=1, col=1, annotation_text=row.name, annotation_angle=45)
#     # fig.add_vline(x=row.cpu_start, line_width=1, line_dash="dot", line_color="red", row=2, col=1, annotation_text=row.name)
#     # use vrect instead
#     fig.add_vrect(x0=row.cpu_start, x1=row.cpu_end, fillcolor="red", opacity=0.1, layer="below", row=1, col=1)
#     fig.add_vrect(x0=row.cpu_start, x1=row.cpu_end, fillcolor="red", opacity=0.1, layer="below", row=2, col=1)






# doesn't work try something else
# add a colorbar to the right of the figure


# add  to fig overlay adapt to px.bar

fig.show()

In [13]:
df.columns

Index(['cpu_start', 'cpu_end', 'name', 'id', 'device_memory_usage', 'stack',
       'module'],
      dtype='object')

In [ ]:
df["module"].apply(lambda x: x.split("/")[1] if len(x.split("/")) > 1 else x).unique()

array(['VisionTransformer_0', 'ResidualMLP_0', 'PARTViT_0'], dtype=object)

In [31]:
evs[0].time_range.start / 1e3

47.030998

In [62]:
# Create a dataframe for memory usage over time with datetime
memory_events = pd.DataFrame(
    [
        {
            "timestamp": ev.time_range.start / 1e3,  # convert to datetime
            # "memory": ev.device_memory_usage,
            # "memory": ev.device_memory_usage / 1024 ** 2,
            "memory": ev.device_memory_usage / 1024 ** 3,
            "name": ev.name
        }
        for ev in evs
        if ev.device_memory_usage is not None
    ]
)

# Sort by timestamp 
memory_events = memory_events.sort_values("timestamp")

# Create cumulative memory usage
memory_events["cumulative_memory"] = memory_events["memory"].cumsum()

# Create the plot
fig = px.area(
    memory_events, 
    x="timestamp",
    y="cumulative_memory",
    title="GPU Memory Usage Over Time",
    labels={
        "timestamp": "Time",
        "cumulative_memory": "Memory Usage (GB)"
    }
)

# add a vertical line at each time a module starts
# for example module PartViT_0/VisonTransformer_0/Sequential_0/Block_6/Attention_6 starts on the ev with minimum ev.time_range.start
# let's keep only PartViT_0/VisonTransformer_0/Sequential_0/Block_6
# df["last_module"]  = df["module"].apply(lambda x: x.split("/")[-1])
# df_grouped_by_module = df.groupby("last_module")
# for module, df_module in df_grouped_by_module:
#     module_start = df_module["cpu_start"].min()
#     # print(f"Module {module} starts at {module_start}")
#     fig.add_vline(
#         x=module_start.timestamp() * 1000,
#         line_dash="dash",
#         line_color="red",
#         annotation_text=module
#     )

df["main_module"] = df["module"].apply(lambda x: x.split("/")[1] if len(x.split("/")) > 1 else x)
df_grouped_by_module = df.groupby("main_module")
for module, df_module in df_grouped_by_module:
    module_start = df_module["cpu_start"].min()
    print(f"Module {module} starts at {module_start}")
    # add a vertical line at each time a module starts



fig.add_hline(
    y=memory_events["cumulative_memory"].max(), 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"Max Memory Usage: {memory_events['cumulative_memory'].max():.2f} GB"
)
fig.update_xaxes(
    title_text="Time",
    tickformat='%L',
    ticklabelmode="period",
    rangeslider_visible=True,
)

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Memory Usage (GB)",
    showlegend=False
)

fig.show()

Module PARTViT_0 starts at 129.012869
Module ResidualMLP_0 starts at 128.383555
Module VisionTransformer_0 starts at 47.030998


In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_memory_usage", row_limit=5))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
     

In [17]:
import torch.autograd.profiler_util
e: torch.autograd.profiler_util.FunctionEvent = prof.events()[100]
# get first event with stack
